<a href="https://colab.research.google.com/github/jh941213/dacon_call/blob/main/dacon_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 호출

In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# 시드(seed) 고정

매번 고정된 결과를 얻기 위해서 사용합니다.  
시드를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나올 수 있습니다.  
항상 동일한 결과를 얻기 위해서 사용합니다.

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 데이터 불러오기
read_csv() 함수를 사용하여 데이터를 불러옵니다.

In [3]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [4]:
# train.csv 파일의 데이터를 확인합니다.
train.head(3)

,ID,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
0,TRAIN_00000,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328.0,32.89,2.0,0.0
1,TRAIN_00001,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209.0,32.32,2.0,0.0
2,TRAIN_00002,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328.0,8.28,0.0,0.0


# 피처엔지니어링


In [ ]:
test

In [5]:
train['총통화시간'] = train['주간통화시간'] + train['저녁통화시간'] + train['밤통화시간']
train['총통화횟수'] = train['주간통화횟수'] + train['저녁통화횟수'] + train['밤통화횟수']
train['총통화요금'] = train['주간통화요금'] + train['저녁통화요금'] + train['밤통화요금']
train['평균통화시간'] = train['총통화시간'] / train['총통화횟수']
train['평균통화요금'] = train['총통화요금'] / train['총통화횟수']
#train['주간통화사용'] = train['주간통화시간'] + train['주간통화횟수']
#train['저녁통화사용'] = train['저녁통화시간'] + train['저녁통화횟수']
#train['밤통화사용'] = train['밤통화시간'] + train['밤통화횟수']
#train['평균통화사용'] = train['주간통화사용'] + train['저녁통화시간'] + train['밤통화사용']
#train['상담빈도횟수'] =train['상담전화건수']/ train['가입일']

In [6]:
test['총통화시간'] = test['주간통화시간'] + test['저녁통화시간'] + test['밤통화시간']
test['총통화횟수'] = test['주간통화횟수'] + test['저녁통화횟수'] + test['밤통화횟수']
test['총통화요금'] = test['주간통화요금'] + test['저녁통화요금'] + test['밤통화요금']
test['평균통화시간'] = test['총통화시간'] / test['총통화횟수']
test['평균통화요금'] = test['총통화요금'] / test['총통화횟수']
#test['주간통화사용'] = test['주간통화시간'] + test['주간통화횟수']
#test['저녁통화사용'] = test['저녁통화시간'] + test['저녁통화횟수']
#test['밤통화사용'] = test['밤통화시간'] + test['밤통화횟수']
#test['평균통화사용'] = test['주간통화사용'] + test['저녁통화시간'] + test['밤통화사용']
#test['상담빈도횟수'] =test['상담전화건수']/ test['가입일']


In [7]:
# 예측에 도움이 될 수 없는 ID 컬럼을 제거합니다.
# x_train은 독립변수이므로 종속변수를 제거합니다.
x_train = train.drop(['ID', '전화해지여부'], axis = 1)

# y_train은 종속변수의 값을 설정해줍니다.
y_train = train['전화해지여부']

# test 데이터에서 예측에 도움이 될 수 없는 ID 컬럼을 제거합니다.
test = test.drop('ID', axis = 1)

In [12]:
x_train.dropna(inplace=True)

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,총통화시간,총통화횟수,총통화요금,평균통화시간,평균통화요금
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328.0,32.89,2.0,630.9,489.0,89.11,1.290184,0.182229
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209.0,32.32,2.0,916.1,858.0,148.11,1.067716,0.172622
2,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328.0,8.28,0.0,770.2,973.0,75.30,0.791572,0.077390
3,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234.0,9.36,0.0,658.2,518.0,58.36,1.270656,0.112664
4,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223.0,2.80,8.0,583.3,514.0,59.58,1.134825,0.115914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15797,28,33,330.5,99,33.7,321.1,208,32.82,204.6,92.0,2.20,2.0,856.2,399.0,68.72,2.145865,0.172231
15798,70,0,213.5,87,37.8,206.4,78,17.40,257.0,101.0,11.55,1.0,676.9,266.0,66.75,2.544737,0.250940
15799,28,0,283.7,83,29.5,287.1,83,22.82,229.5,222.0,2.82,2.0,800.3,388.0,55.14,2.062629,0.142113
15800,88,1,228.7,209,31.3,224.3,92,28.00,283.7,209.0,6.22,8.0,736.7,510.0,65.52,1.444510,0.128471


In [13]:
test.dropna(inplace=True)

In [ ]:
from sklearn.cluster import KMeans
CLUSTERS = [10,50,100,500,1000]


for n_clusters in CLUSTERS:
    # KMeans 클러스터링 모델을 생성합니다
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    # 모델을 훈련합니다
    kmeans.fit(x_train)
    
    # 각 데이터가 속한 클러스터 번호를 예측합니다
    cluster_labels = kmeans.predict(x_train)
    
    # 클러스터링 결과를 저장합니다
    x_train['cluster_'+str(n_clusters)] = cluster_labels


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: F

In [ ]:
from sklearn.cluster import KMeans
CLUSTERS = [10,50,100,500,1000]


for n_clusters in CLUSTERS:
    # KMeans 클러스터링 모델을 생성합니다
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    # 모델을 훈련합니다
    kmeans.fit(test)
    
    # 각 데이터가 속한 클러스터 번호를 예측합니다
    cluster_labels = kmeans.predict(test)
    
    # 클러스터링 결과를 저장합니다
    test['cluster_'+str(n_clusters)] = cluster_labels

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: F

# 주성분 분석

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
temp = train.drop(['ID', '전화해지여부'], axis=1)
temp

In [ ]:
train

In [ ]:
temp_test = test.drop(['ID'], axis=1)
temp

In [ ]:
scaler = StandardScaler()
temp = scaler.fit_transform(temp)
temp

In [ ]:
scaler = StandardScaler()
temp_test = scaler.fit_transform(test)
temp

In [ ]:
pca = PCA()
pca.fit(temp)

In [ ]:
pca_test = PCA()
pca_test.fit(temp_test)

In [ ]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

In [ ]:
sum = 0
for i, ratio in enumerate(pca.explained_variance_ratio_):
    sum += ratio
    print(f'{sum} by PCA{i}')

In [ ]:
sum = 0
for i, ratio in enumerate(pca_test.explained_variance_ratio_):
    sum += ratio
    print(f'{sum} by PCA{i}')

In [ ]:
temp = pca.fit_transform(temp)
temp = pd.DataFrame(temp)
temp

In [ ]:
temp_test = pca_test.fit_transform(temp_test)
temp_test = pd.DataFrame(temp_test)
temp_test

In [ ]:
train = pd.concat([train, temp.iloc[:, 0:8]], axis=1)
train

In [ ]:
train = train.drop(['ID', '전화해지여부'], axis=1)

In [ ]:
train

In [ ]:
test.columns

In [ ]:
test = pd.concat([test, temp_test.iloc[:, 0:8]], axis=1)
test

In [ ]:
train.columns = train.columns.astype(str)
test.columns = train.columns.astype(str)  # 모든 열 이름을 문자열로 변환

In [ ]:
x_train

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,...,주간통화사용,저녁통화사용,밤통화사용,평균통화사용,상담빈도횟수,cluster_10,cluster_50,cluster_100,cluster_500,cluster_1000
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,...,192.2,336.8,590.9,1051.9,0.006079,3,44,45,295,334
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,...,646.9,595.4,531.8,1448.1,1.000000,1,40,23,122,248
2,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328,...,605.4,529.0,608.8,1421.2,0.000000,1,40,23,263,426
3,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234,...,444.4,294.0,437.8,1115.2,0.000000,5,27,34,114,555
4,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223,...,318.3,292.9,486.1,1028.3,0.036036,3,42,45,210,447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30195,263,80,289.6,201,21.8,280.5,323,29.88,208.0,66,...,490.6,603.5,274.0,1045.1,0.007605,6,32,69,166,179
30196,283,81,210.7,280,90.5,284.1,202,32.80,287.8,203,...,490.7,486.1,490.8,1265.6,0.007067,8,49,70,2,801
30197,24,0,222.4,33,22.1,233.9,32,22.22,293.6,95,...,255.4,265.9,388.6,877.9,0.083333,0,3,92,92,586
30198,63,1,262.4,202,29.6,280.6,282,28.88,280.9,207,...,464.4,562.6,487.9,1232.9,0.031746,8,30,20,264,507


# 모델 정의  
의사결정나무 모델을 사용하기 위해 모델을 정의합니다.  
[의사결정나무 모델의 개념을 알고 싶다면 여기를 클릭해주세요.](https://dacon.io/competitions/open/235698/talkboard/403509?page=1&dtype=recent)

In [ ]:
x_train

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,...,총통화시간,총통화횟수,총통화요금,평균통화시간,평균통화요금,주간통화사용,저녁통화사용,밤통화사용,평균통화사용,상담빈도횟수
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,...,630.9,489,89.11,1.290184,0.182229,192.2,336.8,590.9,1051.9,0.006079
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,...,916.1,858,148.11,1.067716,0.172622,646.9,595.4,531.8,1448.1,1.000000
2,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328,...,770.2,973,75.30,0.791572,0.077390,605.4,529.0,608.8,1421.2,0.000000
3,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234,...,658.2,518,58.36,1.270656,0.112664,444.4,294.0,437.8,1115.2,0.000000
4,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223,...,583.3,514,59.58,1.134825,0.115914,318.3,292.9,486.1,1028.3,0.036036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30195,263,80,289.6,201,21.8,280.5,323,29.88,208.0,66,...,778.1,590,60.96,1.318814,0.103322,490.6,603.5,274.0,1045.1,0.007605
30196,283,81,210.7,280,90.5,284.1,202,32.80,287.8,203,...,782.6,685,129.58,1.142482,0.189168,490.7,486.1,490.8,1265.6,0.007067
30197,24,0,222.4,33,22.1,233.9,32,22.22,293.6,95,...,749.9,160,48.54,4.686875,0.303375,255.4,265.9,388.6,877.9,0.083333
30198,63,1,262.4,202,29.6,280.6,282,28.88,280.9,207,...,823.9,691,79.36,1.192330,0.114848,464.4,562.6,487.9,1232.9,0.031746


In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 6.1 MB/s eta 0:00:00


In [8]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 20.0 MB/s eta 0:00:00


In [ ]:

from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

# 모델 파라미터 설정
lgb_params = {'max_depth': 10, 'learning_rate': 0.09,'random_state':42}

cat_params = {'max_depth': 8, 'learning_rate': 0.06800749578049192, 'n_estimators': 967, 'random_state':42}
knn_params = {'n_neighbors':2}
xgb_params = {'max_depth': 10, 'learning_rate': 0.0493607891381531,'random_state':42}
rf_params = {'n_estimators': 697, 'max_depth': 67, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}              
dc_params = {'max_depth': 547, 'min_samples_split': 2, 'min_samples_leaf': 1}
ex_params = {'n_estimators': 489, 'max_depth': 36, 'min_samples_split': 2, 'min_samples_leaf': 1, 'random_state':42}
# 모델 객체 생성
lgb_model = lgb.LGBMClassifier(**lgb_params)
cat_model = CatBoostClassifier(**cat_params)
xgb_model = xgb.XGBClassifier(**xgb_params)
ex_model = ExtraTreesClassifier(**ex_params)
rf_model = RandomForestClassifier(**rf_params)
dc_model = DecisionTreeClassifier(**dc_params)
knn_model = KNeighborsClassifier(**knn_params)

#,('rf',rf_model),('knn',knn_model),('dc',dc_model)
# 스태킹 분류기 객체 생성
estimators = [('lgb', lgb_model),('cat',cat_model), ('xgb', xgb_model), ('ex',ex_model),('dc',dc_model),('rf',rf_model)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=lgb_model)

# k-fold 교차 검증을 수행하여 스태킹 모델 학습
kf = KFold(n_splits=40, shuffle=True, random_state=42)
meta_features = np.zeros((len(x_train),6 ))  # 두번째 레이어 모델의 입력 feature를 저장할 배열
for train_idx, val_idx in kf.split(x_train):
    x_train_fold = x_train.iloc[train_idx]
    y_train_fold = y_train[train_idx]
    x_val_fold = x_train.iloc[val_idx]
    y_val_fold = y_train[val_idx]
    
    # train set을 이용하여 첫번째 레이어 모델 학습
    lgb_model.fit(x_train_fold, y_train_fold)
    cat_model.fit(x_train_fold, y_train_fold)
    xgb_model.fit(x_train_fold, y_train_fold)
    ex_model.fit(x_train_fold, y_train_fold)
    #rf_model.fit(x_train_fold, y_train_fold)
    dc_model.fit(x_train_fold, y_train_fold)
    knn_model.fit(x_train_fold, y_train_fold)

    # validation set에 대한 첫번째 레이어 모델의 예측 확률 계산
    lgb_pred = lgb_model.predict_proba(x_val_fold)[:, 1]
    cat_pred = cat_model.predict_proba(x_val_fold)[:, 1]
    xgb_pred = xgb_model.predict_proba(x_val_fold)[:, 1]
    ex_pred = ex_model.predict_proba(x_val_fold)[:, 1] 
    dc_pred = dc_model.predict_proba(x_val_fold)[:, 1] 
    knn_pred = knn_model.predict_proba(x_val_fold)[:, 1]
    #rf_pred = rf_model.predict_proba(x_val_fold)[:, 1] 
    # 두번째 레이어 모델에 입력할 feature 생성
    meta_features[val_idx, 0] = lgb_pred
    meta_features[val_idx, 1] = cat_pred
    meta_features[val_idx, 2] = xgb_pred
    meta_features[val_idx, 3] = ex_pred
    
    meta_features[val_idx, 4] = dc_pred
    meta_features[val_idx, 5] = knn_pred
    #meta_features[val_idx, 6] = rf_pred
# 두번째 레이어 모델 학습
stacking_model.fit(meta_features, y_train)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
605:	learn: 0.0858101	total: 6.9s	remaining: 4.11s
606:	learn: 0.0855826	total: 6.91s	remaining: 4.1s
607:	learn: 0.0854277	total: 6.92s	remaining: 4.08s
608:	learn: 0.0853208	total: 6.93s	remaining: 4.07s
609:	learn: 0.0852873	total: 6.94s	remaining: 4.06s
610:	learn: 0.0851137	total: 6.95s	remaining: 4.05s
611:	learn: 0.0848912	total: 6.96s	remaining: 4.04s
612:	learn: 0.0847282	total: 6.97s	remaining: 4.03s
613:	learn: 0.0845521	total: 6.98s	remaining: 4.01s
614:	learn: 0.0842314	total: 6.99s	remaining: 4s
615:	learn: 0.0840708	total: 7.01s	remaining: 3.99s
616:	learn: 0.0840030	total: 7.02s	remaining: 3.98s
617:	learn: 0.0838253	total: 7.03s	remaining: 3.97s
618:	learn: 0.0836546	total: 7.04s	remaining: 3.96s
619:	learn: 0.0834883	total: 7.05s	remaining: 3.94s
620:	learn: 0.0832503	total: 7.06s	remaining: 3.93s
621:	learn: 0.0831602	total: 7.07s	remaining: 3.92s
622:	learn: 0.0831052	total: 7.08s	remaining: 3.91s
623:	learn: 0.0829467	total: 7.09

In [8]:
test

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수
0,110,0,210.7,142,35.6,205.9,93,17.43,120.9,111,5.37,2
1,283,1,268.5,100,30.3,283.0,92,28.32,290.0,89,8.82,0
2,34,1,243.3,205,21.3,225.7,205,29.00,92.6,29,2.44,2
3,209,86,83.0,93,24.1,270.6,220,22.89,266.9,81,2.80,6
4,202,0,293.2,23,31.0,294.0,24,26.39,223.8,33,6.66,2
...,...,...,...,...,...,...,...,...,...,...,...,...
12938,322,1,282.4,202,89.9,270.2,321,32.80,293.6,321,8.88,2
12939,222,0,233.5,209,34.6,234.0,92,23.22,224.4,202,9.63,2
12940,98,29,223.4,209,29.3,223.9,203,28.82,230.0,208,22.26,2
12941,23,30,241.3,222,24.7,234.3,93,22.23,223.1,33,9.93,0


In [64]:
# test set에 대한 예측 확률 계산
test_meta_features = np.column_stack((lgb_model.predict_proba(test)[:, 1],
cat_model.predict_proba(test)[:, 1],
xgb_model.predict_proba(test)[:, 1],
ex_model.predict_proba(test)[:, 1],
dc_model.predict_proba(test)[:, 1],
knn_model.predict_proba(test)[:, 1],
knn_model.predict_proba(test)[:, 1]
))
test_preds = stacking_model.predict_proba(test_meta_features)[:, 1]

In [65]:
test_preds

array([0.655942  , 0.00705262, 0.00525879, ..., 0.00205681, 0.09301359,
       0.00323707])

In [66]:
# threshold 값을 설정합니다.
th = 0.3

# test_preds 값이 threshold(th) 이하면 0으로, 그 이상이면 1로 변환합니다.
test_preds_binary = np.where(test_preds <= th, 0, 1)

In [67]:
test_preds_binary

array([1, 0, 0, ..., 0, 0, 0])

#  파일 저장
예측 결과를 제출하기 위해서 예측값을 제출 파일에 할당하고 저장합니다.

In [68]:
# 제출 파일을 불러옵니다.
submit = pd.read_csv('./sample_submission.csv')
# 예측한 값을 '전화해지여부' 컬럼에 할당합니다.


In [69]:
# 예측한 값을 '전화해지여부' 컬럼에 할당합니다.
submit['전화해지여부'] = test_preds_binary
submit.head()

,ID,전화해지여부
0,TEST_00000,1
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,1


In [71]:
# 예측한 파일을 저장합니다.
submit.to_csv('./rf.csv', index = False)